In [ ]:

import os

import math
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from IPython.display import display

from core.live_data_processor import LiveDataLoader
from core.model import Model

In [ ]:
if __name__ == '__main__':
    if not os.path.exists('model'): os.makedirs('model')
    #  Setting pyplot fig size
    plt.rcParams['figure.figsize'] = [10, 8]    
    # Download gold price from yahoo finance
    # gld_dataframe = yf.download(tickers="GLD", start="2000-1-1")
    # Display sample dataframe
    # display(gld_dataframe.head(5))
    # display(gld_dataframe.tail(5))

In [ ]:

    data = LiveDataLoader(
        filename=os.path.join('data', 'gld.csv'),
        split=0.8,
        cols=['Open', 'Close', 'Volume']
    )
    x, y = data.get_train_data(30, True)
    display_x_train = np.reshape(x, (-1, 3))
    display(pd.DataFrame(display_x_train).head(5))
    display_y_train = np.reshape(y, (-1, 1))
    display(pd.DataFrame(display_y_train).head(5))

In [ ]:
    model = Model()
    model.build_model()
    model.model.summary()

In [ ]:
    '''
	# in-memory training
	model.train(
		x,
		y,
		epochs = configs['training']['epochs'],
		batch_size = configs['training']['batch_size'],
		save_dir = configs['model']['save_dir']
	)
	'''
    # out-of memory generative training
    steps_per_epoch = math.ceil(((data.len_train - 30) / 32))
    model.train_generator(
        data_gen=data.generate_train_batch(
            seq_len=30,
            batch_size=32,
            normalise=True
        ),
        epochs=5,
        batch_size=32,
        steps_per_epoch=steps_per_epoch,
        save_file_name='model.h5'
    )    